In [13]:
import os
import pickle
import datetime
import pandas as pd
from src.db import DataBase
from src.utils_stocks import get_df_c

DIR_DB = os.path.join(os.getcwd(), 'data', 'db')
DIR_MODELS = os.path.join(os.getcwd(), 'data', 'models')
db = DataBase([], DIR_DB)

def get_df_proba(df_c, tup_model):
    '''Returns dataframe with only profit predictions
    Args:
        df_c (pandas.DataFrame)
        tup_model (tup):
            q (str): Pandas query string
            ls_col (List of str)
            full_pipe (sklearn.pipeline.Pipeline)
    Returns:
        df_proba (pandas.DataFrame)
    '''
    q, ls_col, full_pipe = tup_model
    # Remove outliers and non-relevant data 
    df = df_c.query(q).copy()
    if df.empty:
        ls_col = ['sym', 'datetime', 'my_index', 'proba', 'datetime_update']
        df_proba = pd.DataFrame(columns=ls_col)
    else:
        s_sym = df['sym']
        s_datetime = df['datetime']
        s_timestamp = [datetime.datetime.now()]*df.shape[0]
        df = df[ls_col]
        arr_proba = full_pipe.predict_proba(df)
        df_proba = pd.DataFrame({
            'sym':s_sym,
            'datetime':s_datetime,
            'my_index':list(df.index),
            'proba':arr_proba[:,1],
            'datetime_update':s_timestamp,
        })
    return df_proba

In [ ]:
f_model = 'tup_model_2021-01-18_2321.p'
with open(os.path.join(DIR_MODELS, f_model), 'rb') as f:
    tup_model = pickle.load(f)

In [17]:
ls_sym = ['TSLA']
date_str = '2021-01-29'
live_data = 0
target_profit, target_loss = 0.01, -0.03
for sym in ls_sym:
    df_c = get_df_c(sym, date_str, live_data, db, target_profit, target_loss)
    df_proba = get_df_proba(df_c, tup_model)
    if not df_proba.empty:
        try:
            idx_signal = df_proba[df_proba['proba']>0.7]['my_index'].values[0]
            idx_buy = idx_signal + 1
            df_c = get_df_c(sym, date_str, live_data, db, target_profit, target_loss)
            df = df_c.iloc[idx_buy:,:]
            df['close'] #to be continued
        except:
            pass
    

,sym,datetime,my_index,proba,datetime_update
22,TSLA,2021-01-29 09:52:00,22,0.601490,2021-01-31 23:25:03.185447
113,TSLA,2021-01-29 11:23:00,113,0.547467,2021-01-31 23:25:03.185447
199,TSLA,2021-01-29 12:49:00,199,0.722000,2021-01-31 23:25:03.185447
